# Labelizer

In [1]:
import pandas as pd

Trabajo con el siguiente rango horario:

In [2]:
firstDay = pd.Timestamp(2019, 4, 18)
lastDay = pd.Timestamp(2019, 4, 21)
secondLastDay = lastDay + pd.Timedelta(3, unit='D')

Los datos son los siguientes:

In [3]:
location = 'data/'
auctionsFile = location + 'auctions.csv'
eventsFile = location + 'events.csv'
clicksFile = location + 'clicks.csv'
installsFile = location + 'installs.csv'

Cargo los datos de los primeros tres dias, y de los segundos tres dias de installs y auctions.

Events y acutions son muy grandes asique los leo de a pedazos

In [4]:
chunksize = 30000

In [5]:
dtype = {
    'date': 'str',
    'ref_type_id': 'category',
    'device_id': 'int64'
}

In [6]:
auctionsIterator = pd.read_csv(auctionsFile, usecols=dtype.keys(), dtype=dtype, parse_dates=['date'], chunksize=chunksize)

In [7]:
auctions = pd.DataFrame()
nextAuctions = pd.DataFrame()
for chunk in auctionsIterator:
    chunkIsInDateInterval = (chunk['date'] >= firstDay) & (chunk['date'] < lastDay)
    chunkIsInSecondDateInterval = (chunk['date'] >= lastDay) & (chunk['date'] < secondLastDay)
    auctions = pd.concat([auctions, chunk[chunkIsInDateInterval]], sort = False)
    nextAuctions = pd.concat([nextAuctions, chunk[chunkIsInSecondDateInterval]], sort = False)

In [8]:
dtype = {
    'date': 'str',
    'ref_type': 'category',
    'ref_hash': 'int64'
}

In [9]:
eventsIterator = pd.read_csv(eventsFile, usecols=dtype.keys(), dtype=dtype, parse_dates=['date'], chunksize=chunksize)

In [10]:
events = pd.DataFrame()
for chunk in eventsIterator:
    chunkIsInDateInterval = (chunk['date'] >= firstDay) & (chunk['date'] < lastDay)
    events = pd.concat([events, chunk[chunkIsInDateInterval]], sort=False)

In [11]:
dtype = {
    'created': 'str',
    'ref_type': 'category',
    'ref_hash': 'int64'
}

In [12]:
installs = pd.read_csv(installsFile, usecols=dtype.keys(), dtype=dtype, parse_dates=['created'])

In [13]:
installsIsInDateInterval = (installs['created'] >= firstDay) & (installs['created'] < lastDay)
installs = installs[installsIsInDateInterval]

In [14]:
installsIsInSecondDateInterval = (installs['created'] >= lastDay) & (installs['created'] < secondLastDay)
nextInstalls = installs[installsIsInSecondDateInterval]

In [15]:
clicks = pd.read_csv(clicksFile, usecols=dtype.keys(), dtype=dtype, parse_dates=['created'])

In [16]:
clicks['created'] = clicks['created'].dt.tz_localize(None)

In [17]:
clicksIsInDateInterval = (clicks['created'] >= firstDay) & (clicks['created'] < lastDay)
clicks = clicks[clicksIsInDateInterval]

Ahora quiero obtener todos los dispositivos que apareceieron en algun momento de los tres dias

In [18]:
devicesInAuctions = pd.Series(auctions['device_id'].unique(), name='device_id')

In [19]:
devicesInInstalls = pd.Series(installs['ref_hash'].unique(), name='device_id')

In [20]:
devicesInClicks = pd.Series(clicks['ref_hash'].unique(), name='device_id')

In [21]:
devicesInEvents = pd.Series(events['ref_hash'].unique(), name='device_id')

In [22]:
devices = pd.concat([devicesInEvents, devicesInClicks, devicesInAuctions, devicesInInstalls], sort=False).unique()
devices = pd.Series(devices)

Ahora quiero conseguir los dispositivos que aparecieron en los segundos 3 dias y los valores de sc y st

In [23]:
st = nextAuctions[['date', 'device_id']].groupby(['device_id']).min()
st['st'] = (st['date'] - lastDay).dt.total_seconds()
st = st.drop(columns=['date'])

In [24]:
sc = nextInstalls[['created', 'ref_hash']].groupby(['ref_hash']).min()
sc['sc'] = (sc['created'] - lastDay).dt.total_seconds()
sc = sc.drop(columns=['created'])

In [25]:
labels = pd.merge(sc, st, how='outer', left_index=True, right_index=True)

Ahora junto los dispositivos que aparecieron en los primeros dias con los labels de los segundos dias, asi desecho aquellos dispositivos que no estaban en los primeros dias

In [26]:
devices.name = 'device_id'

In [27]:
labels = pd.merge(labels, devices, how='right', left_index=True, right_on='device_id')

Por ultimo, reordeno las columnas y guardo el resultado a un csv

In [28]:
labels = labels[['device_id', 'st', 'sc']]

In [29]:
labels.head()

,device_id,st,sc
0,5857744372586891366,NaN,NaN
1,7642521036780133571,436.543534,NaN
2,2548841562898283198,NaN,NaN
3,609402887625919085,NaN,NaN
4,9114651763556439823,NaN,NaN


In [30]:
labels.to_csv('data/labelsFirstTheeDays.csv', index=False)